## **06. Transformasi Data (Data Transformation)**

---
- Mengubah ke refesentasi vektor

- Membagi data ke datavalid, data testing, dan data training

## Import Library

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
import ast

Loading Data

In [11]:
data = pd.read_csv(
    '../05. Pra-pemrosesan Data (Data Preprocessing)/Dataset_Clean/Dataset_Clean(skripsi).csv', 
    sep=","
)

In [12]:
def convert_to_list(text):
    try:
        # Parse the text as a Python literal (list)
        parsed_list = ast.literal_eval(text)
        # Ensure all elements are integers
        return [int(i) for i in parsed_list]
    except (ValueError, SyntaxError):
        return []

data['full_text'] = data['full_text'].apply(convert_to_list)

In [13]:
from gensim.models import Word2Vec

# Train Word2Vec model
sentences = data['full_text'].tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=8, min_count=1, workers=4)

# Retrieve the vocabulary and word vectors
vocab = word2vec_model.wv.key_to_index
word_vectors = word2vec_model.wv.vectors

print(f"Vocabulary size: {len(vocab)}")
print(f"Word vectors shape: {word_vectors.shape}")


ImportError: cannot import name 'Mapping' from 'collections' (c:\Program Files\Python312\Lib\collections\__init__.py)

In [ ]:
import numpy as np

# Function to convert tweet to embedding
def tweet_to_embedding(tweet, model, vector_size):
    embedding = np.zeros((len(tweet), vector_size))
    for i, index in enumerate(tweet):
        if index in model.wv:
            embedding[i] = model.wv[index]
    return embedding

# Example to convert first tweet to embedding
example_tweet = data['full_text'][0]
embedding_example = tweet_to_embedding(example_tweet, word2vec_model, 100)
print(embedding_example)


NameError: name 'word2vec_model' is not defined